In [56]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
import dotenv

dotenv.load_dotenv()

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

In [57]:
class State(MessagesState):
    custom_stuff : str

graph_builder = StateGraph(State)

In [58]:
@tool
def get_weather(city: str):
    """Gets weather in city"""
    return f"The weather in {city} is sunny"

tools = [get_weather]
tools_by_name = {tool.name : tool for tool in tools}
llm_with_tools = llm.bind_tools(tools=tools)

def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages":[response]
    }


In [59]:
from multiprocessing import context
from typing import Literal
from langchain_core.messages import ToolMessage
from langgraph.graph import StateGraph, START, END

def tool_node(state: dict):
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))

    return {"messages":result}

def should_continue(state: MessagesState) -> Literal["tools", END]:

    messages = state["messages"]
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"
    
    return END


In [60]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", should_continue, ["tools", END] )
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile()

graph.invoke(
    {
        "messages":[
            {"role":"user", "content":"what is the weather in machupichu"}
        ]
    }
)

ValueError: Message dict must contain 'role' and 'content' keys, got {'rols': 'user', 'content': 'what is the weather in machupichu'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 